# Earth quake
> graph laplacian

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [Graph Laplacian, PCA]

In [16]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/earthquakes-23k.csv')
df

,Date,Latitude,Longitude,Magnitude
0,01/02/1965,19.2460,145.6160,6.0
1,01/04/1965,1.8630,127.3520,5.8
2,01/05/1965,-20.5790,-173.9720,6.2
3,01/08/1965,-59.0760,-23.5570,5.8
4,01/09/1965,11.9380,126.4270,5.8
...,...,...,...,...
23407,12/28/2016,38.3917,-118.8941,5.6
23408,12/28/2016,38.3777,-118.8957,5.5
23409,12/28/2016,36.9179,140.4262,5.9
23410,12/29/2016,-9.0283,118.6639,6.3


In [4]:
# df.assign 임시 데이터 만들어줘서 원래 데이터 안 건들이게
lst=[df.assign(Year=list(map(lambda x: x.split('/')[-1], df.Date))).\
groupby('Year').\
pipe(list)[i][1].iloc[:,[1,2,3]].\
to_numpy().tolist() for i in range(2017-1965)]

pipe는 튜플로 묶여 있어서 [][] 해줘야 데이터프레임으로 나오지

한 번에 list로 안 되어서 numpy 해준 후 list

`-` 숙제: 

지진자료에서 1985년 이후의 자료만 고른뒤에 HeatMapWithTime()을 이용하여 시각화하라.

In [9]:
lst2=[df.assign(Year=list(map(lambda x: x.split('/')[-1], df.Date))).query('Year>="1985"').\
groupby('Year').\
pipe(list)[i][1].iloc[:,[1,2,3]].\
to_numpy().tolist() for i in range(2016-1985)]

- t = Year
- x = Latitude
- y = Longitude
- f = Magnitude

In [24]:
_df = df.assign(Year=list(map(lambda x: x.split('/')[-1], df.Date))).query('Year>="2010"').reset_index().iloc[:,2:]
_df

,Latitude,Longitude,Magnitude,Year
0,12.4240,141.9490,6.1,2010
1,-5.2970,145.8590,5.8,2010
2,-8.8020,-77.7180,5.7,2010
3,-8.7260,157.4870,6.6,2010
4,-8.7830,157.3540,7.1,2010
...,...,...,...,...
3569,38.3917,-118.8941,5.6,2016
3570,38.3777,-118.8957,5.5,2016
3571,36.9179,140.4262,5.9,2016
3572,-9.0283,118.6639,6.3,2016


In [30]:
np.abs(_df.Longitude[0]-_df.Longitude[1])

3.910000000000025

$$dist(i,i+1)$$

In [74]:
def dist(i):
    return np.abs(_df.Longitude[i]-_df.Longitude[i+1])

In [75]:
dist(0)

3.910000000000025

In [76]:
dist(1)

223.577

$$W_{i,j}=\begin{cases}{exp(-\frac{[dist(i,j)]^2}{2\theta^2})} & \quad \text{if } dist(i,j) \leq κ \\ 0 & \quad \text{otherwise} \end{cases},   (1)$$

In [81]:
def weight(i,theta=1,kappa=100):
    return np.exp( -dist(i)**2 / (2*theta**2) ) if dist(i) <= kappa else 0

In [78]:
weight(0)

0.00047888503490364315

In [79]:
weight(1)

0

In [80]:
np.exp(-dist(0)*2 / (2*1*2) )

0.14156447669413225